In [9]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env")

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi I'm hamin, I live in South Korea", "Wow that is so cool!")

In [2]:
memory.load_memory_variables({"input": "who is hamin"})

{'history': [SystemMessage(content='On hamin: hamin lives in South Korea.')]}

In [3]:
add_message("hamin likes kimchi", "Wow that is so cool!")

In [4]:
memory.load_memory_variables({"inputs": "what does hamin like"})

{'history': [SystemMessage(content='On hamin: hamin lives in South Korea. hamin likes kimchi.')]}

In [8]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import sqlite3  # or replace with another RDB of choice

load_dotenv(dotenv_path="../.env")

llm = ChatOpenAI(temperature=0.1)

class RDBConversationKGMemory(ConversationKGMemory):
    def __init__(self, db_path, **kwargs):
        # Initialize the database connection
        self.connection = sqlite3.connect("./conversation.db")
        # Call superclass initializer
        super().__init__(**kwargs)
        # Create table if it doesn't exist
        self.create_table()

    def create_table(self):
        with self.connection:
            self.connection.execute('''CREATE TABLE IF NOT EXISTS conversation (
                                       id INTEGER PRIMARY KEY,
                                       input TEXT,
                                       output TEXT,
                                       timestamp DATETIME DEFAULT CURRENT_TIMESTAMP)''')

    def save_context(self, inputs, outputs):
        input_text = inputs["input"]
        output_text = outputs["output"]
        with self.connection:
            self.connection.execute(
                "INSERT INTO conversation (input, output) VALUES (?, ?)",
                (input_text, output_text)
            )

    def load_memory_variables(self):
        with self.connection:
            # Retrieve and format stored messages
            return [{"input": row[0], "output": row[1]} for row in
                    self.connection.execute("SELECT input, output FROM conversation ORDER BY timestamp")]

memory = RDBConversationKGMemory(db_path="conversation.db", llm=llm, return_messages=True)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

# Example usage
add_message("Hi I'm hamin, I live in South Korea", "Wow that is so cool!")

ValueError: "RDBConversationKGMemory" object has no field "connection"